In [16]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
from utils import process_runner_books

# Change this certs path to wherever you're storing your certificates
certs_path = "/home/naing/certs/"

# Change these login details to your own
my_username = "mightytranks@gmail.com"
my_password = "OsGiwuevzRf4925Lfty!"
my_app_key = "XzTt9hf8DAsmOuUa" #live

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certs_path)

trading.login()

## TODO, place bet only using these three arguments
venueName = 'henlow'
myRaceID = '1.200269396'
liability_amount = 30

In [17]:
# Create a price filter. Get all traded and offer data
price_filter = filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# Request market books
market_books = trading.betting.list_market_book(
    market_ids=[myRaceID],
    price_projection=price_filter
)

# Grab the first market book from the returned list as we only requested one market 
market_book = market_books[0]

runners_df = process_runner_books(market_book.runners)
runners_df



,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor
0,41419585,15.5,14.53,18.5,6.56,15.5,28.94,ACTIVE,None,None
1,41419586,20.0,36.03,25.0,40.40,23.0,109.68,ACTIVE,None,None
2,41419587,15.0,23.07,18.0,12.39,14.5,165.24,ACTIVE,None,None
3,41419588,34.0,23.18,48.0,6.83,34.0,3.53,ACTIVE,None,None
4,41419589,27.0,23.80,34.0,11.69,27.0,20.97,ACTIVE,None,None
5,41419592,19.0,8.39,21.0,8.64,18.0,71.78,ACTIVE,None,None
6,41419593,27.0,14.56,36.0,10.41,29.0,117.36,ACTIVE,None,None
7,41419594,22.0,15.37,26.0,10.79,22.0,5.61,ACTIVE,None,None
8,41419595,48.0,12.62,70.0,5.46,46.0,14.61,ACTIVE,None,None
9,41419596,36.0,20.22,60.0,20.06,36.0,7.54,ACTIVE,None,None


In [18]:
#TODO: extract a list of IDs for the lay options
lay_options_ids = []
for obj in market_book.runners:
    lay_options_ids.append(obj.selection_id)

print(lay_options_ids)

[41419585, 41419586, 41419587, 41419588, 41419589, 41419592, 41419593, 41419594, 41419595, 41419596, 41419599, 41419600, 41419601, 41419602, 41419603, 41419606, 41419607, 41419608, 41419609, 41419610, 41419613, 41419614, 41419615, 41419616, 41419617, 41419620, 41419621, 41419622, 41419623, 41419624]


In [19]:
from utils import choose_lay_option

fav_selection_index = choose_lay_option(venueName)

if(fav_selection_index == -1):
    print("ERROR!!! 2 same odds found")
    
elif(fav_selection_index == -2):
    print("ERROR!!! less than 6 runners")

else:
    print("Lay option found successfully")
    print(fav_selection_index)

fav_selection_id = lay_options_ids[fav_selection_index]
print(fav_selection_id)
fav_price = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Best Lay Price']

limit_order_filter = filters.limit_order(
    price=fav_price,
    persistence_type='LAPSE',
    bet_target_type='PAYOUT',
    bet_target_size=liability_amount # use this if i want $1500 as liability
)

instructions_filter = filters.place_instruction(
    selection_id = str(fav_selection_id),
    order_type = "LIMIT",
    side="LAY",
    limit_order=limit_order_filter
)

instructions_filter

['3.70', '4.40', '4.50', '2.90', '11.00', '6.50']
['2.90', '3.70', '4.40', '4.50', '6.50', '11.00']
4,1
15
41419606


{'orderType': 'LIMIT',
 'selectionId': '41419606',
 'side': 'LAY',
 'limitOrder': {'price': 13.0,
  'persistenceType': 'LAPSE',
  'betTargetType': 'PAYOUT',
  'betTargetSize': 1500}}

In [20]:
order = trading.betting.place_orders(
    market_id = myRaceID,
    customer_strategy_ref='Naing_maker',
    instructions=[instructions_filter]
)

In [21]:
# listClearedOrders
cleared_orders = trading.betting.list_cleared_orders(bet_status="SETTLED",
                                                    market_ids=[myRaceID])


# Create a DataFrame from the orders
pd.DataFrame(cleared_orders._data['clearedOrders'])

,eventTypeId,eventId,marketId,selectionId,handicap,betId,placedDate,persistenceType,orderType,side,betOutcome,priceRequested,settledDate,lastMatchedDate,betCount,priceMatched,priceReduced,sizeSettled,profit,customerStrategyRef
0,4339,31534256,1.200269396,41419606,0.0,271775862591,2022-06-18T15:07:36.000Z,LAPSE,LIMIT,LAY,WON,13.0,2022-06-18T15:10:32.000Z,2022-06-18T15:07:36.000Z,1,12.99,False,115.45,115.45,Naing_maker
